# Consumo de agua CDMX

## Parameters

In [1]:
# BASE_DIR = '/Users/efraflores/Desktop/EF/Contests/Datathon_CDMX/data' # Mac
BASE_DIR = r'D:\Users\efras\Documents\EF\Contests\Datathon_CDMX\data' # Windows
# RESOURCE_ID = '932b56bf-c5ec-4815-9814-370d58754002' #####SÓLO TRAE INFO 2016
FILE_NAME = 'aborto_legal'

## Class

In [18]:
from pandas import DataFrame
from typing import Dict
from numpy import nan


from mariachis.models import BaseClass

from pandas import cut
from sklearn.preprocessing import OneHotEncoder

class AbortoLegal(BaseClass):
    def __init__(self, base_dir: str, file_name: str) -> None: 
        super().__init__(base_dir, file_name)

    def wrangling_aborto(self, df: DataFrame, clean_dict: Dict, date_col: str='fingreso', export_imputed: bool=True, **kwargs):
        # Sólo registros con fecha
        df = df[df[date_col].notnull()].reset_index()
        # Crear variables de fecha
        df = self.date_vars(df, date_col)

        # Obtener las variables numéricas
        vars_num = list(set(
            clean_dict['vars_numbin']+
            list(clean_dict['vars_num'].keys())
        ))
        
        # Limpiar si hay texto de variables numéricas
        for col in vars_num:
            df[col] = df[col].map(self.clean_number).astype(float)
        
        # Función para convertir 1.0 --> 01
        def two_char(n):
            return str(int(n)).zfill(2)

        # Crear rangos de variables numéricas
        for col, to_group in clean_dict['vars_num'].items():
            # Encontrar el bin al cual el dato pertenece
            df[f'rango_{col}'] = cut(df[col], bins=[-1]+to_group+[1000])
            # Convertirlo a texto: [1.0 - 5.0] --> '01 a 05'
            df[f'rango_{col}'] = df[f'rango_{col}'].map(lambda x: two_char(x.left+1)+' a '+two_char(x.right) if x!=nan else nan)
            # Corregir algunas etiquetas como: '01 a 01' --> '01' y también '03 a 1000' --> '> 02'
            df[[f'rango_{col}']] = df[[f'rango_{col}']].replace({
                **{two_char(to_group[-1]+1)+' a 1000': '> '+two_char(to_group[-1])},
                **{two_char(x)+' a '+two_char(x): two_char(x) for x in to_group}
            })
            # No perder de vista los valores ausentes. "La falta de información también es información"
            df[f'rango_{col}'] = df[f'rango_{col}'].map(lambda x: nan if str(x)=='nan' else str(x))
        
        # Sólo saber si son mayores a 0 o no, tomar en cuenta que NaN > 0 --> False
        for col in clean_dict['vars_numbin']:
            df[col] = df[col].map(lambda x: '> 0' if x>0 else x)

        # Obtener las variables que serán binarias
        vars_cat = list(set(
            clean_dict['vars_first_word']+
            list(clean_dict['vars_cat'].keys())+
            clean_dict['vars_yes_no']+
            clean_dict['vars_just_fill_na']
        ))

        # Omitir acentos de variables categóricas
        for col in vars_cat:
            df[col] = df[col].map(self.clean_text)

        # Obtener la primer palabra
        for col in clean_dict['vars_first_word']:
            df[col] = df[col].str.split().str[0]

        # Agrupar categorías
        for col,to_group in clean_dict['vars_cat'].items():
            df[col] = df[col].map(to_group)

        cluster_cols = vars_cat+[f'rango_{col}' for col in clean_dict['vars_num'].keys()]+clean_dict['vars_numbin']
        
        # Lo que quedo vacío, marcar como "DESCONOCIDO"
        for col in cluster_cols:
            df[col] = df[col].fillna('DESCONOCIDO').astype(str)

        # Tal vez el usuario quiere exportar los resultados
        if export_imputed: self.export_csv(df, name_suffix='imputed')

        # Crear una columna por clase para todas las variables, que ahora son categóricas
        ohe = OneHotEncoder().fit(df[cluster_cols])
        X = DataFrame(ohe.transform(df[cluster_cols]).toarray(), columns=ohe.get_feature_names_out())

        # Obtener grupos 
        X, cluster_pipe = self.make_clusters(X, cluster_obj='')
        if 1==1: return X

        # Estructurar la información limpia
        # df = df.pivot_table(index=['fingreso_yearmonth', 'entidad', 'alc_o_municipio'])
        # df.columns = ['_'.join([x for x in col]) if not isinstance(df.columns[0],str) else col for col in df.columns]
        return df

al = AbortoLegal(BASE_DIR, FILE_NAME)
# print(al)

## Data Wrangling

In [3]:
df = al.full_import(api=False)
df.sample()

,año,mes,fingreso,autoref,edocivil_descripcion,edad,desc_derechohab,nivel_edu,ocupacion,religion,...,tanalgesico,cconsejo,panticoncep,fecha_cierre,resultado_ile,medicamento,clues_2,clues_hospital,procile_simplificada,procile_completa
23873,2018.0,OCTUBRE,NaN,NO,SOLTERA,27.0,NO ESPECIFICADO,LICENCIATURA,NaN,CATOLICA,...,SI,SI,NaN,NaN,COMPLETA,NaN,DFSSA003705,DFSSA003705,MEDICAMENTO,NaN


## Dict

In [5]:
# Diccionario para aplicar limpieza por tipo de variable
full_dict = {}

# Ahora, limpiar texto de variables numéricas
full_dict['vars_numbin'] = ['naborto', 'npartos', 'ncesarea', 'nile']
full_dict['vars_num'] = {
    'edad': [9,11,20,22,25,28,32],
    'menarca': [0,11,12,13,14],
    'fsexual': [0,15,16,17,18,19],
    'sememb': [0,5,6,7,8,9],
    'nhijos': [0,1,2],
    'gesta': [0,1,2,3,4],
    'c_num': [0,1,2],
    'p_semgest': [0,5,6,7,8,9],
    'p_diasgesta': [0,1,2,3,4,5]
}

# Obtener la primer palabra
full_dict['vars_first_word'] = ['anticonceptivo',  'panticoncep']

# Binarias de sí o no
full_dict['vars_yes_no'] = ['autoref', 'consejeria', 'p_consent', 's_complica', 'c_dolor', 'tanalgesico', 'cconsejo', 'resultado_ile']

# Sólo cambiar nulos por "DESCONOCIDO"
full_dict['vars_just_fill_na'] = ['entidad', 'motiles', 'desc_servicio', 'anticonceptivo', 'panticoncep']

# Agrupar categorías
full_dict['vars_cat'] = {
    'edocivil_descripcion':{
        'SOLTERA':'SOLTERX', 
        'UNION LIBRE':'UNION_LIBRE', 
        'CASADA':'CASADX',
        'DIVORCIADA':'SEPARADX',
        'SEPARADA':'SEPARADX',
        'VIUDA':'SEPARADX',
    },
    'ocupacion':{
        'EMPLEADA':'EMPLEADX',
        'ESTUDIANTE':'ESTUDIANTE',
        'TRABAJADORA DEL HOGAR NO REMUNERADA':'TRAB_HOGAR_NO_REMUNERADX',
        'DESEMPLEADA':'DESEMPLEADX',
    },
    'desc_derechohab':{
        'NINGUNO':'NINGUNO',
        'IMSS':'ALGUNO',
        'SEGURO POPULAR':'ALGUNO',
        'OTRA':'ALGUNO',
        'ISSSTE':'ALGUNO',
    },
    'nivel_edu':{
        'LICENCIATURA':'LICENCIATURA O MAYOR',
        'MAESTRIA':'LICENCIATURA O MAYOR',
        'DOCTORADO':'LICENCIATURA O MAYOR',
        'PREPARATORIA':'PREPARATORIA O MENOR',
        'SECUNDARIA':'PREPARATORIA O MENOR',
        'PRIMARIA':'PREPARATORIA O MENOR',
        'SIN ACCESO A LA EDUCACION FORMAL':'PREPARATORIA O MENOR',
    },
    'parentesco':{
        'PAREJA':'PAREJA',
        'EX-PAREJA':'PAREJA',
        'FAMILIAR MUJER':'FAMILIAR',
        'FAMILIAR HOMBRE':'FAMILIAR',
        'FAMILIAR SIN ESPECIFICAR':'FAMILIAR',
        'TUTOR O RESPONSABLE LEGAL':'FAMILIAR',
        'PERSONA CERCANA':'AMIGX',
        'PERSONA CERCANA MUJER':'AMIGX',
        'PERSONA CERCANA HOMBRE':'AMIGX',
        'AMIGA':'AMIGX',
        'AMIGO':'AMIGX',
        'CONOCIDA':'OTRX',
        'CONOCIDO':'OTRX',
        'RELACION LABORAL':'OTRX',
        'OTRA':'OTRX',
    },
    'religion':{
        'NINGUNA':'NINGUNA',
        'CATOLICA':'SI',
        'CRISTIANA':'SI',
        'OTRA':'SI',
        'MORMONA':'SI',
        'TESTIGA DE JEHOVA':'SI',
        'BUDISTA':'SI',
        'JUDIA':'SI',
        'ANGLICANA':'SI',
        'MUSULMANA':'SI',
    },
}

In [ ]:
from pandas import qcut, cut

aux_dict = {}
for col in full_dict['vars_num']:
    new_col = df[col].map(al.clean_number).astype(float)
    orig_bins = qcut(new_col, q=6, retbins=True, duplicates='drop')[-1]
    new_bins = [orig_bins[0]-1] + list(orig_bins[:-1]) + [1e3]
    aux_dict[col] = new_bins

aux_dict

## Transform

In [19]:
X = al.wrangling_aborto(df, full_dict)
X.sample(4)

,c_dolor_DESCONOCIDO,c_dolor_NO,c_dolor_SI,autoref_DESCONOCIDO,autoref_NO,autoref_SI,s_complica_DESCONOCIDO,s_complica_NO,s_complica_SI,resultado_ile_COMPLETA,...,naborto_DESCONOCIDO,npartos_0.0,npartos_> 0,npartos_DESCONOCIDO,ncesarea_0.0,ncesarea_> 0,ncesarea_DESCONOCIDO,nile_0.0,nile_> 0,nile_DESCONOCIDO
10774,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
73865,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
74417,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
69672,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0


In [15]:
ohe = OneHotEncoder().fit(X.head())
X = DataFrame(ohe.transform(X.head()).toarray(), columns=ohe.get_feature_names_out())
X

,c_dolor_c_dolor_DESCONOCIDO,autoref_autoref_DESCONOCIDO,s_complica_s_complica_DESCONOCIDO,resultado_ile_COMPLETA,resultado_ile_OTRO,consejeria_NO,consejeria_SI,parentesco_parentesco_DESCONOCIDO,edocivil_descripcion_CASADX,edocivil_descripcion_SOLTERX,...,rango_p_semgest_00,rango_p_semgest_01 a 05,rango_p_semgest_06,rango_p_diasgesta_rango_p_diasgesta_DESCONOCIDO,naborto_0.0,naborto_> 0,npartos_0.0,ncesarea_0.0,ncesarea_> 0,nile_nile_DESCONOCIDO
0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,...,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0
1,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,...,0.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0
2,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,...,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0
3,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,...,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0
4,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,...,1.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0


In [ ]:
df.isnull().mean()

Siguientes pasos!
- numéricas a rangos, para definir a los nulos como nueva categoría
- clustering añomes+municipio con vars_cat+vars_catbin escaladas y calculando ['count','min','mean','max']
- perfilamiento
- agrupar por municipio, cómo es la distribución de los clústeres previos?
- Cómo vender lo que resuelve?